In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA_ext.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA_ext.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined_ext.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined_ext.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA_ext.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA__ext.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA_ext.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("MA_Apr_weekend_ext");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
demandsDict = readall("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_weekend/results/demandsDictFixed1.json");
demandsDict = JSON.parse(demandsDict);

In [6]:
# demands = demandsDict["$(length(demandsDict)-1)"]

demands = demandsDict

Dict{UTF8String,Any} with 484 entries:
  "(16,1)"  => 0.0
  "(11,20)" => 77.46554012209326
  "(2,8)"   => 0.0
  "(3,22)"  => 0.0
  "(20,1)"  => 0.0
  "(14,5)"  => 0.0
  "(19,5)"  => 0.0
  "(11,1)"  => 0.0
  "(9,6)"   => 0.0
  "(2,17)"  => 0.0
  "(15,20)" => 73.18127123342225
  "(8,12)"  => 0.0
  "(22,20)" => 0.0
  "(8,9)"   => 9.269549717518359
  "(14,11)" => 3.425299799596956
  "(15,22)" => 0.0
  "(13,16)" => 470.61705551061226
  "(19,2)"  => 0.0
  "(2,18)"  => 0.0
  "(11,17)" => 0.0
  "(11,7)"  => 0.0
  "(3,3)"   => 0.0
  "(15,21)" => 13.57008000222226
  "(4,1)"   => 409.9983112770102
  "(16,3)"  => 0.0
  ⋮         => ⋮

In [7]:
odPairLabel_;

In [8]:
demandsVec = zeros(462)

for i = 1:length(demandsVec)
    demandsVec[i] = demands["($(odPairLabel_["$i"][1]),$(odPairLabel_["$i"][2]))"]
end

In [9]:
demandsVec

462-element Array{Float64,1}:
  30.7515
 187.255 
 287.624 
   0.0   
 261.377 
  86.4444
   0.0   
   0.0   
   0.0   
   0.0   
 274.937 
   0.0   
   0.0   
   ⋮     
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   

In [10]:
for key=keys(odPairRoute)
    if contains(key, "462-")
        println(key)
    end
end

462-329


In [11]:
linkRoute

Dict{UTF8String,Any} with 1729 entries:
  "24-106" => 1.0
  "42-74"  => 1.0
  "42-365" => 1.0
  "60-8"   => 1.0
  "73-260" => 1.0
  "22-355" => 1.0
  "19-180" => 1.0
  "62-421" => 1.0
  "68-199" => 1.0
  "2-47"   => 1.0
  "66-86"  => 1.0
  "50-450" => 1.0
  "7-225"  => 1.0
  "32-306" => 1.0
  "40-13"  => 1.0
  "39-243" => 1.0
  "1-225"  => 1.0
  "41-157" => 1.0
  "28-23"  => 1.0
  "70-83"  => 1.0
  "40-1"   => 1.0
  "50-273" => 1.0
  "8-28"   => 1.0
  "6-291"  => 1.0
  "57-195" => 1.0
  ⋮        => ⋮

In [12]:
deg = 6
c = 0.5
lam = 10000.0

coeffs_dict = readall("../temp_files/coeffs_dict_Apr_weekend.json")
coeffs_dict = JSON.parse(coeffs_dict)
fcoeffs = coeffs_dict["($(deg),$(c),$(lam),1)"]

# fcoeffs = [1, 0, 0, 0, 0, 0, 0, 0, 0]

7-element Array{Any,1}:
  1.0       
  0.00268857
  0.0114664 
 -0.00834401
  0.0234205 
 -0.0137301 
  0.00429399

In [13]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 74
numRoute = 4614
numOD = 462

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

@defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
    free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
    free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6, a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2265
Number of nonzeros in inequality constraint Jacobian.:     4614
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:     4688
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [14]:
getValue(linkFlow)

74-element Array{Float64,1}:
  693.453     
 1094.09      
  448.632     
 2945.37      
  298.658     
 2159.65      
  944.538     
 1667.93      
  491.379     
  791.602     
   35.4982    
   37.6282    
   -5.02041e-9
    ⋮         
  940.339     
  892.708     
  802.509     
 1393.21      
   -9.49996e-8
   -9.99996e-9
   -9.49996e-8
   -9.99996e-9
  351.459     
  687.401     
  638.695     
  478.043     

In [15]:
getObjectiveValue(m)

8097.785815973345

In [16]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 74 entries:
  (17,15) => 687.4013836643727
  (3,6)   => 491.37871267149467
  (21,13) => 1038.3016567234538
  (22,16) => -9.999959819928062e-9
  (19,12) => -1.999991964305247e-8
  (22,14) => -9.499961830277479e-8
  (15,17) => 351.45876586824306
  (10,11) => 421.92344778342357
  (13,12) => 0.0
  (3,1)   => 2945.369124707012
  (16,13) => 892.7079553775958
  (4,5)   => 828.2450940454478
  (13,16) => 940.3394445857888
  (17,16) => 478.0425555830145
  (20,13) => 925.9678063503316
  (12,8)  => 385.9003695948744
  (2,4)   => 944.5377707237731
  (16,17) => 638.6950688229305
  (1,2)   => 693.4531884365739
  (14,21) => 829.5914871290009
  (2,3)   => 298.65819618447307
  (13,19) => 12.051888555161101
  (13,14) => 0.0
  (16,14) => 1393.2130447323577
  (8,12)  => 873.5609038783274
  ⋮       => ⋮

In [17]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekend_ext

In [18]:
flow_user = GLS_Apr_weekend_ext.x_

74x30 Array{Float64,2}:
 1366.35    791.95    771.324  …  1015.01        1624.03     280.272
 1206.76    992.338   991.158     1000.59        1273.13     570.559
 2114.24   1499.68   1200.6       1171.78        1608.83     947.273
 2273.84   1299.3     980.768     1186.21        1959.73     656.986
   58.465   201.97    271.021       86.5748        40.0646   261.282
  139.064   247.675   201.826  …    23.5801       146.69     143.578
 1193.7    1571.81   1353.56       781.132       1109.21    1362.48 
  953.514  1726.49   1642.59       829.701        651.679   1770.47 
 2124.9    1398.79    938.436     1533.92        1629.54    1007.64 
 2365.09   1244.1     649.406     1485.35        2087.07     599.646
  406.406   514.047   567.011  …   469.666        455.737    655.187
  281.997  1677.2    1591.06       204.042        239.965   1660.44 
  204.522   246.959  1020.38         1.2451e-13   268.26    1131.08 
    ⋮                          ⋱                                    
  817.703 

In [19]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
        free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
        free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
        free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + 
        free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + 
        free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [20]:
weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

for i in weekend_Apr_list
    println(socialObj(flow_user[:, i])/getObjectiveValue(m))
end

1.1031084747780133
0.8528166089135252
1.2710336880796254
0.7736200889352572
1.1092363627794535
0.7611861294427611
1.0770833909241309
0.7730301302101971
0.8953988022852275


In [21]:
length(weekend_Apr_list)

9